In [117]:
import pandas as pd
import matplotlib.pyplot as plt
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion, nms_method

In [118]:
# # HTC results for WBF
# sources = [
#            "/opt/ml/temp/ensemble/source/output_HJH_6187.csv",
#            "/opt/ml/temp/ensemble/source/output_HJH_5999.csv"
#           ]

# # cascade results for WBF
# sources = [
#            "/opt/ml/temp/ensemble/source/output_HJH_6109.csv",
#            "/opt/ml/temp/ensemble/source/output_HJH_5781.csv"
#           ]

# # faster_rcnn for WBF
# sources = [
#            "/opt/ml/temp/ensemble/source/output_KJS_5594.csv",
#            "/opt/ml/temp/ensemble/source/output_KJS_5139.csv"
#           ]

# # retinanet for WBF
# sources = [
#            "/opt/ml/temp/ensemble/source/output_KJS_5531.csv",
#            "/opt/ml/temp/ensemble/source/output_KJS_5779.csv",
#            "/opt/ml/temp/ensemble/source/output_KJS_5830.csv"
#           ]

# # yolov8x for WBF
# sources = [
#            "/opt/ml/temp/ensemble/source/output_KDH_4590.csv",
#            "/opt/ml/temp/ensemble/source/output_KDH_4536.csv"
#           ]

# wbf results for nms
sources = [
           "/opt/ml/temp/ensemble/source/ensemble_wbf_cascade.csv",
           "/opt/ml/temp/ensemble/source/ensemble_wbf_fasterrcnn.csv",
           "/opt/ml/temp/ensemble/source/ensemble_wbf_HTC.csv",
           "/opt/ml/temp/ensemble/source/ensemble_wbf_retinanet.csv",
           "/opt/ml/temp/ensemble/source/ensemble_wbf_yolov8x.csv"
          ]

In [119]:
df_list = []
for source in sources:
    df_list.append(pd.read_csv(source))
print(df_list)

[                                       PredictionString       image_id
0     7 0.9761859774589539 602.17431640625 518.15820...  test/0000.jpg
1     5 0.9827036261558533 343.0611572265625 251.796...  test/0001.jpg
2     1 0.9778867363929749 75.7828369140625 272.6979...  test/0002.jpg
3     9 0.8574692606925964 25.2581787109375 200.2873...  test/0003.jpg
4     7 0.40000802278518677 210.1573486328125 398.90...  test/0004.jpg
...                                                 ...            ...
4866  1 0.9150110483169556 294.3243408203125 348.559...  test/4866.jpg
4867  3 0.9758195281028748 416.169921875 314.4879150...  test/4867.jpg
4868  7 0.9283799529075623 117.8038330078125 418.506...  test/4868.jpg
4869  0 0.7913827896118164 113.1009521484375 231.381...  test/4869.jpg
4870  4 0.980180025100708 374.177734375 205.45812988...  test/4870.jpg

[4871 rows x 2 columns],                                        PredictionString       image_id
0     7 0.9837279319763184 603.10595703125 517.883

In [120]:
prediction_strings = []
file_names = []

conf_thr = 0.1

for image_id in range(4871):
    # print(f'{image_id:04d}')
    labels_list = []
    scores_list = []
    bboxes_list = []

    isValidCount = 0
    
    for df in df_list:
        df_cell = df.iloc[image_id]["PredictionString"]
        # print(df_cell)
        
        if type(df_cell) is str:
            isValidCount += 1
            annotations_raw = df_cell.split()
            annotations = [annotations_raw[i:i + 6] for i in range(0, len(annotations_raw), 6)]
            
            labels = []
            scores = []
            bboxes = []

            for annotation in annotations:
                category = int(annotation[0])
                confidence = float(annotation[1])
                bbox = [float(s) / 1024 for s in annotation[2:]]

                labels.append(category)
                scores.append(confidence)
                bboxes.append(bbox)
    
            labels_list.append(labels)
            scores_list.append(scores)
            bboxes_list.append(bboxes)

    if isValidCount < len(df_list):
        print(f'image_id: {image_id}, isValidCount: {isValidCount} / {len(df_list)}')

    # ensemble 
    weights = [1 for _ in range(isValidCount)]
    iou_thr = 0.50#0.62#0.7
    skip_box_thr = conf_thr#0.0001
    sigma = 0.1

    # boxes, scores, labels = nms(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)    
    # boxes, scores, labels = soft_nms(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    # boxes, scores, labels = non_maximum_weighted(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    # boxes, scores, labels = weighted_boxes_fusion(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

    # boxes, scores, labels = weighted_boxes_fusion(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes, scores, labels = nms_method(bboxes_list, scores_list, labels_list, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr, weights=weights)

    # submission file content
    prediction_string = ''
    for label, score, box in zip(labels, scores, boxes):
        if box[0] < 0.0: box[0] = 0.0
        elif box[0] > 1.0: box[0] = 1.0
        
        if box[1] < 0.0: box[1] = 0.0
        elif box[1] > 1.0: box[1] = 1.0
        
        if box[2] < 0.0: box[2] = 0.0
        elif box[2] > 1.0: box[2] = 1.0
        
        if box[3] < 0.0: box[3] = 0.0
        elif box[3] > 1.0: box[3] = 1.0
        
        prediction_string += str(int(label))+ ' ' + str(score) + ' ' + str(box[0] * 1024) + ' ' + str(box[1] * 1024) + ' ' + str(box[2] * 1024) + ' ' + str(box[3] * 1024) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(f'test/{image_id:04d}.jpg')

image_id: 44, isValidCount: 0 / 2
image_id: 109, isValidCount: 1 / 2
image_id: 137, isValidCount: 1 / 2
image_id: 358, isValidCount: 1 / 2
image_id: 395, isValidCount: 1 / 2
image_id: 585, isValidCount: 1 / 2
image_id: 695, isValidCount: 1 / 2
image_id: 709, isValidCount: 1 / 2
image_id: 730, isValidCount: 1 / 2
image_id: 882, isValidCount: 1 / 2
image_id: 953, isValidCount: 1 / 2
image_id: 1119, isValidCount: 1 / 2
image_id: 1135, isValidCount: 1 / 2
image_id: 1208, isValidCount: 0 / 2
image_id: 1291, isValidCount: 1 / 2
image_id: 1376, isValidCount: 1 / 2
image_id: 1474, isValidCount: 0 / 2
image_id: 1497, isValidCount: 0 / 2
image_id: 1528, isValidCount: 0 / 2
image_id: 1875, isValidCount: 1 / 2
image_id: 1950, isValidCount: 1 / 2
image_id: 2119, isValidCount: 1 / 2
image_id: 2150, isValidCount: 0 / 2
image_id: 2168, isValidCount: 1 / 2
image_id: 2352, isValidCount: 0 / 2
image_id: 2614, isValidCount: 1 / 2
image_id: 2755, isValidCount: 1 / 2
image_id: 2816, isValidCount: 0 / 2
imag

In [121]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./ensemble.csv', index=None)